In [42]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [43]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [44]:
train, dev = train_test_split(df, test_size=0.1, random_state=42)

In [45]:
train.shape, dev.shape

((1080, 8), (120, 8))

In [46]:
df['rating'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [47]:
categories = ['B', 'A', 'A-', 'A+', 'B-', 'B+']

In [48]:
dev = dev.query('rating in @categories')

In [49]:
train = train.query('rating in @categories')

In [50]:
train.shape, dev.shape

((389, 8), (41, 8))

In [51]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [52]:
nlp = spacy.load('ru_core_news_lg')

In [53]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [54]:
create_docbin(train).to_disk("train.spacy")
create_docbin(dev).to_disk("dev.spacy")

In [55]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       29.95    0.30
  0     200          27.10       51.83    0.52
  1     400          19.04       60.93    0.61
  1     600          11.10       60.56    0.61
  2     800           9.52       65.83    0.66
  2    1000           5.00       60.30    0.60
  3    1200           5.36       60.75    0.61
  3    1400           2.95       66.23    0.66
  4    1600           3.04       61.03    0.61
  4    1800           2.38       65.31    0.65
  5    2000           1.63       66.66    0.67
  5    2200           1.00       67.68    0.68
  6    2400           1.80       65.80    0